In [202]:
import pandas as pd
from glob import glob
import re
import os


In [203]:
# 全行全列表示
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
# 小数点以下n桁表示
pd.options.display.float_format = '{:.2f}'.format

In [204]:
csv_files = glob('../csv/EXA_FIRST_マイジャグラーV_*.csv')
len(csv_files), csv_files[0]

(152, '../csv\\EXA_FIRST_マイジャグラーV_2024-08-29.csv')

In [205]:
def extract_information(csv_file):
    pattern = r"(.+)_(.+)_(\d{4}-\d{2}-\d{2}).csv"

    file_name = os.path.basename(csv_file)
    # print(file_name)
    match = re.match(pattern, file_name)

    if match:
        hall_name, machine_name, date_str = match.groups()
        return hall_name, machine_name, date_str
    else:
        print(f"ファイル名の形式が一致しません {file_name}")
        print(f"ホール名: {hall_name}")
        print(f"機種名: {machine_name}")
        print(f"日付: {date_str}")


In [206]:
def grape_rate(game_count, bb_count, rb_count, coins):
    bb_coins = 240
    rb_coins = 96
    cherry_coins = 2
    cherry_rate = 38.1
    replay_coins = 3
    replay_rate = 7.3
    grape_coins = 8

    in_coins = game_count * 3

    out_with_cherry = (
        bb_coins * bb_count
        + rb_coins * rb_count
        + (cherry_coins * game_count / cherry_rate)
        + (replay_coins * game_count / replay_rate)
    )
    without_cherry_rate = 0.667
    out_without_cherry = (
        bb_coins * bb_count
        + rb_coins * rb_count
        + (cherry_coins * game_count / cherry_rate * without_cherry_rate)
        + (replay_coins * game_count / replay_rate)
    )

    grape_rate_with_cherry = (
        grape_coins * game_count / (coins + in_coins - out_with_cherry)
    )
    grape_rate_without_cherry = (
        grape_coins * game_count / (coins + in_coins - out_without_cherry)
    )

    # print("フリー打ち : ", grape_rate_without_cherry)
    # print("チェリー狙い : ", grape_rate_with_cherry)
    
    return grape_rate_with_cherry, grape_rate_without_cherry


# game_count = 8413
# bb_count = 33
# rb_count = 36
# coins = 1862

# grape_rate(game_count, bb_count, rb_count, coins)

In [207]:
dataframes = []
for csv_file in csv_files:
    hall_name, machine_name, date_str = extract_information(csv_file)
    df = pd.read_csv(csv_file, encoding="utf-8-sig")
    df["date"] = date_str
    df["hall_name"] = hall_name
    df["machine_name"] = machine_name
    dataframes.append(df)
    
if dataframes:
    df = pd.concat(dataframes, ignore_index=True)
    df.to_csv('../analysis_data.csv', index=False)

df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12312 entries, 0 to 12311
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   台番号           12312 non-null  int64  
 1   G数            12312 non-null  object 
 2   差枚            12312 non-null  object 
 3   BB            12312 non-null  int64  
 4   RB            12231 non-null  float64
 5   合成確率          12312 non-null  object 
 6   BB確率          12312 non-null  object 
 7   RB確率          12231 non-null  object 
 8   date          12312 non-null  object 
 9   hall_name     12312 non-null  object 
 10  machine_name  12312 non-null  object 
dtypes: float64(1), int64(2), object(8)
memory usage: 1.0+ MB


In [208]:
df["G数"] = (
    df["G数"]
    .astype(str)  # 文字列型に変換
    .str.replace(",", "", regex=True)  # カンマを削除
    .str.extract(r"(\d+)")  # 数字部分のみ抽出
    .dropna()  # NaNを削除
    .astype(int)  # 整数型に変換
)
df["差枚"] = (
    df["差枚"]
    .astype(str)  # 文字列型に変換
    .str.replace(",", "", regex=True)  # カンマを削除
    .str.extract(r"(\d+)")  # 数字部分のみ抽出
    .dropna()  # NaNを削除
    .astype(int)  # 整数型に変換
)
df["合成確率"] = (
    df["合成確率"]
    .astype(str)  # 文字列型に変換
    .str.replace("1/", "", regex=True)  # カンマを削除
    .str.extract(r"(\d+)")  # 数字部分のみ抽出
    .dropna()  # NaNを削除
    .astype(float)  # 整数型に変換
)
df["BB確率"] = (
    df["BB確率"]
    .astype(str)  # 文字列型に変換
    .str.replace("1/", "", regex=True)  # カンマを削除
    .str.extract(r"(\d+)")  # 数字部分のみ抽出
    .dropna()  # NaNを削除
    .astype(float)  # 整数型に変換
)
df["RB確率"] = (
    df["RB確率"]
    .astype(str)  # 文字列型に変換
    .str.replace("1/", "", regex=True)  # カンマを削除
    .str.extract(r"(\d+)")  # 数字部分のみ抽出
    .dropna()  # NaNを削除
    .astype(float)  # 整数型に変換
)

df["date"] = pd.to_datetime(df["date"], errors="coerce")
df["month"] = df["date"].dt.month.astype(int)
df['weekday'] = df['date'].dt.weekday.astype(int)
df['day'] = df['date'].dt.day.astype(int)
df["last_digit_of_day"] = df["date"].dt.day.astype(str).str[-1].astype(int)
df["g_cherry"], df["g_free"] = grape_rate(df["G数"], df["BB"], df["RB"], df["差枚"])

df.to_csv('../analysis_data.csv', index=False)


In [209]:
df.head()

,台番号,G数,差枚,BB,RB,合成確率,BB確率,RB確率,date,hall_name,machine_name,month,weekday,day,last_digit_of_day,g_cherry,g_free
0,1001,9238,2268,39,40.00,116.00,236.00,230.00,2024-08-29,EXA_FIRST,マイジャグラーV,8,3,29,9,5.91,5.84
1,1002,794,185,2,1.00,264.00,397.00,794.00,2024-08-29,EXA_FIRST,マイジャグラーV,8,3,29,9,3.91,3.88
2,1003,4433,609,17,19.00,123.00,260.00,233.00,2024-08-29,EXA_FIRST,マイジャグラーV,8,3,29,9,5.96,5.88
3,1004,3384,1347,16,16.00,105.00,211.00,211.00,2024-08-29,EXA_FIRST,マイジャグラーV,8,3,29,9,5.94,5.87
4,1005,2926,474,10,6.00,182.00,292.00,487.00,2024-08-29,EXA_FIRST,マイジャグラーV,8,3,29,9,4.76,4.71


In [217]:
target_df = df[df["day"] == 29]
game_count = target_df.pivot_table(index="台番号", values="G数", aggfunc="sum")
coin_count = target_df.pivot_table(index="台番号", values="差枚", aggfunc="sum")
bb_count = target_df.pivot_table(index="台番号", values="BB", aggfunc="sum")
rb_count = target_df.pivot_table(index="台番号", values="RB", aggfunc="sum")
df_day = pd.concat([coin_count, game_count, bb_count, rb_count], axis=1)
df_day['BB_rate'] = df_day['G数'] / df_day["BB"]
df_day['RB_rate'] = df_day['G数'] / df_day["RB"]
df_day['TOTAL_rate'] = df_day['G数'] / (df_day["RB"]+df_day["BB"])
df_day[(df_day["RB_rate"] <= 300) & (df_day["TOTAL_rate"] <= 150)].sort_values(["RB_rate", "TOTAL_rate"])

,差枚,G数,BB,RB,BB_rate,RB_rate,TOTAL_rate
台番号,,,,,,,
1019,10641,35823,154,138.00,232.62,259.59,122.68
1087,5050,26539,92,98.00,288.47,270.81,139.68
1003,3532,26119,92,96.00,283.90,272.07,138.93
1098,6841,20613,82,73.00,251.38,282.37,132.99
1040,4715,24730,86,87.00,287.56,284.25,142.95
1001,9101,31022,104,107.00,298.29,289.93,147.02
1099,6288,34857,119,120.00,292.92,290.48,145.85
1077,8079,40932,159,140.00,257.43,292.37,136.90
1013,6407,18189,71,62.00,256.18,293.37,136.76


In [211]:
df[(df["台番号"] == 1019) & (df["day"] == 29)]

,台番号,G数,差枚,BB,RB,合成確率,BB確率,RB確率,date,hall_name,machine_name,month,weekday,day,last_digit_of_day,g_cherry,g_free
18,1019,7990,2638,36,36.00,111.00,221.00,221.00,2024-08-29,EXA_FIRST,マイジャグラーV,8,3,29,9,5.91,5.84
99,1019,4114,156,14,9.00,178.00,293.00,457.00,2024-09-29,EXA_FIRST,マイジャグラーV,9,6,29,9,5.17,5.11
342,1019,9369,4959,49,39.00,106.00,191.00,240.00,2024-10-29,EXA_FIRST,マイジャグラーV,10,1,29,9,5.67,5.60
2853,1019,4735,535,19,15.00,139.00,249.00,315.00,2024-11-29,EXA_FIRST,マイジャグラーV,11,4,29,9,5.79,5.71
5202,1019,1202,491,3,3.00,200.00,400.00,400.00,2024-12-29,EXA_FIRST,マイジャグラーV,12,6,29,9,3.80,3.77
7713,1019,8413,1862,33,36.00,121.00,254.00,233.00,2025-01-29,EXA_FIRST,マイジャグラーV,1,2,29,9,5.69,5.62
